In [1]:
import os
import numpy as np
from keras.preprocessing import image
from keras.applications import densenet
import keras
import pickle
from scipy.spatial.distance import cosine
from PIL import Image


In [2]:
model = densenet.DenseNet201(include_top=False, weights='imagenet', input_shape=(256, 256, 3), pooling='avg', classes=350)

In [3]:
def getencodings(path):    
    _img = image.load_img(path, target_size=(256, 256))
    _img = keras.utils.img_to_array(_img)
    _img = np.expand_dims(_img, axis=0)
    _enc = densenet.preprocess_input(_img)
    _enc = model.predict(_enc)
    return _enc

In [4]:
type(getencodings("C:/Users/Aarush Raj/OneDrive/Desktop/img2rec/DEEP-CHEF-PROJECT/downloaded_images/train/0_Chicken Tikka Masala/1_Chicken Tikka Masala.jpg"))

1/1 ━━━━━━━━━━━━━━━━━━━━ 31s 31s/step


numpy.ndarray

In [5]:
if __name__ == '__main__':
    names_list=os.listdir(r"C:\Users\Aarush Raj\OneDrive\Desktop\img2rec\DEEP-CHEF-PROJECT\downloaded_images\train")
    encodings_list = []
    recipe_list=[]
    c = 0
    for name in names_list[0:5]:
        recipe_name=name.split("_")[1]
        for number in range(1,9):
            encoding=[]
            image_path="C:/Users/Aarush Raj/OneDrive/Desktop/img2rec/DEEP-CHEF-PROJECT/downloaded_images/train/"+name+"/"+str(number)+"_"+recipe_name+".jpg"
            encoding = getencodings(image_path)
            encodings_list.append(encoding)
            recipe_list.append(recipe_name)
        c += 1
    print(len(recipe_list), len(encodings_list))

    with open('encodings.txt', 'wb') as file:
        pickle.dump(encodings_list, file)
    with open('enc_names.txt', 'wb') as file:
        pickle.dump(recipe_list, file)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 952ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 313ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 292ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 178ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 163ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 228ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 274ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 172ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 174ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 213ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 285ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 255ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 180ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 199ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 255ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 227ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 259ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 203ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 236ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 245ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 293ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 188ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 293ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 163ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 269ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 

In [7]:
len(recipe_list)

40

In [26]:
from chardet import detect
with open(r"C:\Users\Aarush Raj\OneDrive\Desktop\img2rec\DEEP-CHEF-PROJECT\aarush\enc_names.txt","rb") as f:
    file=f.read()
    print(file.decode('Windows-1252'))
    #print(detect(file)['encoding'])
    

€•ª       ]”(ŒChicken Tikka Masala”hhhhhhhŒIndian Fry Bread”hhhhhhhŒShrimp Vindaloo”hhhhhhhŒMasala chai”hhhhhhhŒBasmati Pulao nuts”hhhhhhhe.


In [ ]:
type(encodings_list)

list

In [ ]:
image_path

'C:/Users/Aarush Raj/OneDrive/Desktop/img2rec/DEEP-CHEF-PROJECT/downloaded_images/train/0_Chicken Tikka Masala/1_Chicken Tikka Masala.jpg'

just checking

In [ ]:
_img = image.load_img(r"C:\Users\Aarush Raj\OneDrive\Desktop\img2rec\DEEP-CHEF-PROJECT\downloaded_images\test\1_Naan\9_Naan.jpg", target_size=(256, 256))
_img = keras.utils.img_to_array(_img)
_img = np.expand_dims(_img, axis=0)
_enc = densenet.preprocess_input(_img)
_enc = model.predict(_enc)
_enc

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 193ms/step


array([[9.7090713e-05, 7.5560465e-04, 1.5090167e-03, ..., 2.7459392e-01,
        1.9984721e-01, 1.2217317e-01]], dtype=float32)

In [ ]:
print(os.listdir(r"C:\Users\Aarush Raj\OneDrive\Desktop\img2rec\DEEP-CHEF-PROJECT\downloaded_images"))

['hello', 'test', 'train']


just checking

In [ ]:
img = image.load_img(r"C:\Users\Aarush Raj\OneDrive\Desktop\img2rec\DEEP-CHEF-PROJECT\downloaded_images\train\1_Naan\2_Naan.jpg", target_size=(256, 256))
img = keras.utils.img_to_array(img)
img = np.expand_dims(img, axis=0)
enc = densenet.preprocess_input(img)
enc = model.predict(enc)
print(enc)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 291ms/step
[[9.1914306e-05 5.8829499e-04 1.2086974e-03 ... 7.5686830e-01
  6.8234848e-03 7.4211019e-04]]


In [ ]:
img1 = image.load_img(r"C:\Users\Aarush Raj\OneDrive\Desktop\img2rec\DEEP-CHEF-PROJECT\downloaded_images\train\1_Naan\5_Naan.jpg", target_size=(256, 256))
img1 = keras.utils.img_to_array(img1)
img1 = np.expand_dims(img1, axis=0)
enc1 = densenet.preprocess_input(img1)
enc1 = model.predict(enc1)
print(enc1)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 222ms/step
[[1.3076351e-04 3.2538065e-04 1.4036527e-03 ... 5.8270401e-01
  1.1677598e-02 9.1031656e-02]]


In [ ]:
similarity=cosine(_enc[0],enc[0])
s1=cosine(_enc[0],enc1[0])
s2=cosine(enc[0],enc1[0])
s3=cosine(enc1[0],enc[0])
print(similarity,s1,s2,s3)

0.38520339248235214 0.39427060395575675 0.38686255295183825 0.38686255295183825


In [ ]:
img7 = image.load_img(r"C:\Users\Aarush Raj\OneDrive\Desktop\img2rec\DEEP-CHEF-PROJECT\downloaded_images\train\1_Naan\7_Naan.jpg", target_size=(256, 256))
img7 = keras.utils.img_to_array(img7)
img7 = np.expand_dims(img7, axis=0)
enc7 = densenet.preprocess_input(img7)
enc7 = model.predict(enc7)
print(enc7)

img3 = image.load_img(r"C:\Users\Aarush Raj\OneDrive\Desktop\img2rec\DEEP-CHEF-PROJECT\downloaded_images\train\1_Naan\3_Naan.jpg", target_size=(256, 256))
img3 = keras.utils.img_to_array(img3)
img3 = np.expand_dims(img3, axis=0)
enc3 = densenet.preprocess_input(img3)
enc3 = model.predict(enc3)
print(enc3)

print(cosine(enc3[0],enc7[0]))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 177ms/step
[[6.5320186e-05 4.3773686e-04 1.4675669e-03 ... 1.0696392e+00
  2.8892964e-02 3.2143086e-02]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 288ms/step
[[1.1759455e-04 4.8694725e-04 1.9708984e-03 ... 1.1663135e+00
  9.2119738e-02 1.7724106e-02]]
0.16504474276782277


In [ ]:
print(type(_enc))
print(len(_enc[0]))
print(len(enc[0]))
print(_enc[0][156])

<class 'numpy.ndarray'>
1920
1920
1.6126285e-05


In [ ]:
with open("enc_names.txt","r") as f:
    d=f.readline()
    print(d)

€•þ$      ]”(Œ0_Chicken Tikka Masala”Œ100_Indian Fry Bread”Œ101_Shrimp Vindaloo”Œ102_Masala chai”Œ103_Basmati Pulao nuts”Œ"104_Butternut Squash Coconut Curry”Œ105_Aloo Matar”Œ106_Ground Lamb”Œ107_Indian Spiced Omelet”Œ108_Palak Paneer”Œ109_Chicken Tandoori”Œ10_Chicken Rice and Spices Bake”Œ110_Garlic Naan”Œ111_Chai Oatmeal”Œ	112_Kheer”Œ113_Garlic Naan”Œ114_Spicy Lentil Soup”Œ115_Indian Beef Madras Curry”Œ116_Cauliflower Curry”Œ117_Biryani”Œ118_Samosas”Œ119_Cashew Chicken Fingers”Œ11_Indian Butter Chicken”Œ120_Rogan Josh Stew”Œ121_Phall curry”Œ12_Curry Chicken”Œ



In [4]:
import json
import pandas as pd
df=pd.read_json(r"C:\Users\Aarush Raj\OneDrive\Desktop\img2rec\DEEP-CHEF-PROJECT\aarush\recipes.json")
df1=df.query('name=="Naan"')
df1

,name,cooking_time,calories,ingredients,directions,serving
1,Naan,40mins,213.1,4 cups flour\n1 teaspoon baking powder\n1 t...,"Mix together flour, baking powder and salt.\r\...",10
20,Naan,40mins,213.1,4 cups flour\n1 teaspoon baking powder\n1 t...,"Mix together flour, baking powder and salt.\r\...",10
367,Naan,8mins,213.9,4 ounces refrigerated bread dough (see Rec...,Use the refrigerated pre-mixed dough for No-Kn...,1


In [6]:
import os
lis=os.listdir(r"C:\Users\Aarush Raj\OneDrive\Desktop\img2rec\DEEP-CHEF-PROJECT\downloaded_images\train")
rec=[x.split("_")[0] for x in lis]
print(lis[0:5])
print(rec[0:5])


['0_Chicken Tikka Masala', '100_Indian Fry Bread', '101_Shrimp Vindaloo', '102_Masala chai', '103_Basmati Pulao nuts']
['0', '100', '101', '102', '103']
